In [1]:
from msgs import MAVLink_heartbeat_message

In [3]:
# __init__(self, type: int, autopilot: int, base_mode: int, custom_mode: int, system_status: int, mavlink_version: int)
MAV_TYPE = 2
MAV_AUTOPILOT = 1
MAV_MODE = 0
MAV_STATE = 0
MAVLINK_VERSION = 1
CRC_EXTRA_HEARTBEAT = 50
CRC_EXTRA_COMMAND_LONG = 20
CRC_EXTRA_COMMAND_ACK = 143

msg = MAVLink_heartbeat_message(type=MAV_TYPE, autopilot=MAV_AUTOPILOT, base_mode=MAV_MODE, custom_mode=MAV_MODE, system_status=MAV_STATE, mavlink_version=MAVLINK_VERSION)
for i in range(1):
    print(msg.pack())

b'\xfe\t\x00\x01\x01\x00\x00\x00\x00\x00\x02\x01\x00\x00\x01\x89R'


# Testing decoder

In [4]:
from decoder import decoder

dec = decoder()
# known msgs ------------------------>
# from_msg = b'\xfe!\x13\xff\xbeL\x00\x00\x14C\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x01\x01\x00Q-'

# unknown msgs ------------------------>
# from_msg = b'\xfe\x0b\x16\xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01\xb2\x82'
# from_msg = b'\xfe\x0b \xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01hT'
from_msg = b'\xfe\x0b\xe5\xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01\r@'



for byte in from_msg:
    msg = dec.parse_char(bytes([byte]))  # Преобразуем каждый элемент в байт
    if msg:
        # Проверяем тип сообщения
        print(msg)
        if msg.get_type() == "MANUAL_CONTROL":
            # Извлекаем значения полей
            x = msg.x
            y = msg.y
            z = msg.z
            print(f"x: {x}, y: {y}, z: {z}")


MANUAL_CONTROL {target : 1, x : 0, y : 0, z : 500, r : 0, buttons : 0}
x: 0, y: 0, z: 500


In [ ]:
import struct
temp = bytearray(b'\xfe\x0b\xe5\xff\xbeE')
struct.unpack("<cBBBBB", temp)
# temp.decode('utf-8')

In [5]:
from pymavlink import mavutil

# Создаем объект MAVLink
master = mavutil.mavlink.MAVLink(None)  # None, потому что мы не используем поток данных

# Сообщение в байтах
from_msg = b'\xfe\x0b\xe5\xff\xbeE\x00\x00\x00\x00\xf4\x01\x00\x00\x00\x00\x01\r@'

# Парсинг сообщения
for byte in from_msg:
    msg = master.parse_char(bytes([byte]))  # Преобразуем каждый элемент в байт
    if msg:
        # Проверяем тип сообщения
        if msg.get_type() == "MANUAL_CONTROL":
            # Извлекаем значения полей
            x = msg.x
            y = msg.y
            z = msg.z
            print(f"x: {msg.x}, y: {msg.y}, z: {msg.z}, r: {msg.r}, buttons : {msg.buttons}")

x: 0, y: 0, z: 500, r: 0, buttons : 0


# Sending mpu datas to qground 

In [2]:
import math
import time

class ComplementaryFilter:
    def __init__(self, alpha=0.999):
        self.alpha = alpha
        self.roll = 0.0
        self.pitch = 0.0
        self.yaw = 0.0
        self.last_time = time.time()
        self.init_roll = None
        self.init_pitch = None
        self.init_yaw = None

    def wrap_to_pi(self, angle):
        return (angle + math.pi) % (2 * math.pi) - math.pi

    def update(self, accel, gyro, mag):
        current_time = time.time()
        dt = current_time - self.last_time
        self.last_time = current_time

        # Вычисление углов на основе акселерометра
        accel_roll = math.atan2(accel.y, accel.z)
        accel_pitch = math.atan2(-accel.x, math.sqrt(accel.y**2 + accel.z**2))

        # Вычисление изменения углов на основе гироскопа
        gyro_roll = gyro.x * dt
        gyro_pitch = gyro.y * dt
        gyro_yaw = gyro.z * dt

        # Комплементарный фильтр для roll и pitch
        self.roll = self.alpha * (self.roll + gyro_roll) + (1 - self.alpha) * accel_roll
        self.pitch = self.alpha * (self.pitch + gyro_pitch) + (1 - self.alpha) * accel_pitch

        # Рассчет yaw на основе магнитометра
        mag_x = mag.x * math.cos(self.pitch) + mag.y * math.sin(self.roll) * math.sin(self.pitch) + mag.z * math.cos(self.roll) * math.sin(self.pitch)
        mag_y = mag.y * math.cos(self.roll) - mag.z * math.sin(self.roll)
        self.yaw = math.atan2(mag_y, mag_x)

        # Угловые скорости
        rollspeed = gyro.x  # уже в радианах/сек
        pitchspeed = gyro.y  # уже в радианах/сек
        yawspeed = gyro.z  # уже в радианах/сек

        # Инициализация углов (зафиксировать стартовые значения)
        if self.init_roll is None and self.init_pitch is None and self.init_yaw is None:
            self.init_roll = self.roll
            self.init_pitch = self.pitch
            self.init_yaw = self.yaw

        # Приведение углов к диапазону [-pi, pi] относительно инициализации
        self.roll = self.wrap_to_pi(self.roll - self.init_roll)
        self.pitch = self.wrap_to_pi(self.pitch - self.init_pitch)
        self.yaw = self.wrap_to_pi(self.yaw - self.init_yaw)

        return self.roll, self.pitch, self.yaw, rollspeed, pitchspeed, yawspeed
